In [1]:
print("======== START ETL PARTS9 ========")

======== START ETL PARTS9 ========


In [2]:
import sys
print(sys.executable)

C:\ProgramData\anaconda3\python.exe


In [3]:
BRANCH = "hq"

In [4]:
!pip install sqlalchemy pyodbc psycopg2-binary

In [5]:
#!git clone https://github.com/pthengtr/kcw-analytics.git

In [1]:
import os
os.chdir(r"C:\Users\Windows 11\Notebook\kcw-analytics")
!git pull origin main

Already up to date.


From https://github.com/pthengtr/kcw-analytics
 * branch            main       -> FETCH_HEAD


In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [3]:
DB_PASSWORD = os.getenv("DB_PASSWORD")

In [7]:
import sys
import importlib

repo_path = r"C:\Users\Windows 11\Notebook\kcw-analytics"

if repo_path not in sys.path:
    sys.path.append(repo_path)

import src.kcw.supabase_utils as supabase_utils
importlib.reload(supabase_utils)

add_sales_quality_flags = supabase_utils.add_sales_quality_flags
enrich_sales_with_last_purchase_cost = supabase_utils.enrich_sales_with_last_purchase_cost
qc_unknown = supabase_utils.qc_unknown
refill_last_cost_from_icmas = supabase_utils.refill_last_cost_from_icmas
build_all_dims = supabase_utils.build_all_dims
bulk_upsert_via_stage = supabase_utils.bulk_upsert_via_stage
filter_last_year_from_latest = supabase_utils.filter_last_year_from_latest

In [8]:
from sqlalchemy import create_engine
import pandas as pd
import urllib.parse

server = r'KSS'
database = 'PARTS9'
username = "python_reader"
password = "12345678" 

odbc_str = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"UID={username};"
    f"PWD={password};"
    "TrustServerCertificate=yes;"
)

engine_kss = create_engine("mssql+pyodbc:///?odbc_connect=" + urllib.parse.quote_plus(odbc_str))


In [9]:
def read_latest_year(table_name, date_column="BILLDATE"):
    query = f"""
    SELECT *
    FROM dbo.{table_name}
    WHERE {date_column} >= DATEADD(YEAR, -1,
        (SELECT MAX({date_column}) FROM dbo.{table_name})
    )
    """
    return pd.read_sql(query, engine_kss)

df_raw_hq_sidet = read_latest_year("SIDET")
df_raw_hq_pidet = read_latest_year("PIDET")
df_raw_hq_simas = read_latest_year("SIMAS")
df_raw_hq_pimas = read_latest_year("PIMAS")
df_hq_icmas = pd.read_sql("SELECT * FROM dbo.ICMAS;", engine_kss)

In [4]:
import psycopg2

conn = psycopg2.connect(
    host="aws-0-ap-southeast-1.pooler.supabase.com",   # paste yours here
    port=5432,
    dbname="postgres",
    user="postgres.jdzitzsucntqbjvwiwxm",        # note the dot + project ref
    password=DB_PASSWORD,
    sslmode="require",
)
print("Connected via pooler ✅")

Connected via pooler ✅


In [11]:
# HQ SIDET
result = bulk_upsert_via_stage(
    conn,
    df_raw_hq_sidet,
    target_table=f"raw_{BRANCH}_sidet_lines",
    pk_cols=["ID"],
)

print(result)

{'rows_in_df': 193214, 'target_table': 'public.raw_hq_sidet_lines', 'stage_table': 'public.raw_hq_sidet_lines_stage'}


In [12]:
# HQ SIMAS
result = bulk_upsert_via_stage(
    conn,
    df_raw_hq_simas,
    target_table=f"raw_{BRANCH}_simas_bills",
    pk_cols=["ID"],
)

print(result)

{'rows_in_df': 61839, 'target_table': 'public.raw_hq_simas_bills', 'stage_table': 'public.raw_hq_simas_bills_stage'}


In [13]:
# HQ PIDET
result = bulk_upsert_via_stage(
    conn,
    df_raw_hq_pidet,
    target_table=f"raw_{BRANCH}_pidet_lines",
    pk_cols=["ID"],
)

print(result)

{'rows_in_df': 39376, 'target_table': 'public.raw_hq_pidet_lines', 'stage_table': 'public.raw_hq_pidet_lines_stage'}


In [14]:
# HQ PIMAS
result = bulk_upsert_via_stage(
    conn,
    df_raw_hq_pimas,
    target_table=f"raw_{BRANCH}_pimas_bills",
    pk_cols=["ID"],
)

print(result)

{'rows_in_df': 10925, 'target_table': 'public.raw_hq_pimas_bills', 'stage_table': 'public.raw_hq_pimas_bills_stage'}


In [15]:
# HQ ICMAS
result = bulk_upsert_via_stage(
    conn,
    df_hq_icmas,
    target_table=f"raw_{BRANCH}_icmas_products",
    pk_cols=["ID"],
)

print(result)

{'rows_in_df': 114887, 'target_table': 'public.raw_hq_icmas_products', 'stage_table': 'public.raw_hq_icmas_products_stage'}


In [16]:
from sqlalchemy import create_engine

DATABASE_URL = (
    "postgresql+psycopg2://"
    f"postgres.jdzitzsucntqbjvwiwxm:{DB_PASSWORD}@"
    "aws-0-ap-southeast-1.pooler.supabase.com:5432/postgres"
)

engine_supabase = create_engine(DATABASE_URL)

# test connection
with engine_supabase.connect() as conn_test:
    print("Connected via SQLAlchemy ✅")

Connected via SQLAlchemy ✅


In [17]:
import pandas as pd

df_syp_sidet = pd.read_sql(
    "SELECT * FROM raw_syp_sidet_lines;",
    engine_supabase
)
df_syp_simas = pd.read_sql(
    "SELECT * FROM raw_syp_simas_bills;",
    engine_supabase
)
df_syp_pidet = pd.read_sql(
    "SELECT * FROM raw_syp_pidet_lines;",
    engine_supabase
)
df_syp_pimas = pd.read_sql(
    "SELECT * FROM raw_syp_pimas_bills;",
    engine_supabase
)

df_hq_sidet = pd.read_sql(
    "SELECT * FROM raw_hq_sidet_lines;",
    engine_supabase
)
df_hq_simas = pd.read_sql(
    "SELECT * FROM raw_hq_simas_bills;",
    engine_supabase
)
df_hq_pidet = pd.read_sql(
    "SELECT * FROM raw_hq_pidet_lines;",
    engine_supabase
)
df_hq_pimas = pd.read_sql(
    "SELECT * FROM raw_hq_pimas_bills;",
    engine_supabase
)



In [18]:
data = {}

data['raw_syp_pimas_purchase_bills.csv'] = df_syp_pimas.copy()
data['raw_syp_pidet_purchase_lines.csv'] = df_syp_pidet.copy()
data['raw_syp_sidet_sales_lines.csv'] = df_syp_sidet.copy()
data['raw_syp_simas_sales_bills.csv'] = df_syp_simas.copy()

data['raw_hq_pimas_purchase_bills.csv'] = df_hq_pimas.copy()
data['raw_hq_pidet_purchase_lines.csv'] = df_hq_pidet.copy()
data['raw_hq_sidet_sales_lines.csv'] = df_hq_sidet.copy()
data['raw_hq_simas_sales_bills.csv'] = df_hq_simas.copy()

data['raw_hq_icmas_products.csv'] = df_hq_icmas.copy()

In [19]:
hq_sales_lines = data['raw_hq_sidet_sales_lines.csv'].copy()
syp_sales_lines = data['raw_syp_sidet_sales_lines.csv'].copy()
purchase_lines = data['raw_hq_pidet_purchase_lines.csv'].copy()

In [20]:
syp_sales_flagged = add_sales_quality_flags(syp_sales_lines)

# QC summary (no deletion)
total = len(syp_sales_flagged)
invalid = (~syp_sales_flagged["IS_VALID"]).sum()
print(f"Invalid: {invalid:,}/{total:,} ({invalid/total*100:.2f}%)")

print(
    syp_sales_flagged["INVALID_REASON"]
    .fillna("OK")
    .value_counts()
    .head(20)
)

# For analytics (optional): just filter in pandas
syp_sales_valid_only = syp_sales_flagged[syp_sales_flagged["IS_VALID"]].copy()

Invalid: 13/35,926 (0.04%)
INVALID_REASON
OK            35913
BAD_BCODE         7
BAD_AMOUNT        6
Name: count, dtype: Int64


In [21]:
hq_sales_flagged = add_sales_quality_flags(hq_sales_lines)

# QC summary (no deletion)
total = len(hq_sales_flagged)
invalid = (~hq_sales_flagged["IS_VALID"]).sum()
print(f"Invalid: {invalid:,}/{total:,} ({invalid/total*100:.2f}%)")

print(
    hq_sales_flagged["INVALID_REASON"]
    .fillna("OK")
    .value_counts()
    .head(20)
)

# For analytics (optional): just filter in pandas
syp_sales_valid_only = hq_sales_flagged[hq_sales_flagged["IS_VALID"]].copy()

Invalid: 2,627/356,483 (0.74%)
INVALID_REASON
OK                                353856
CANCELED                            1428
BAD_BCODE                            761
BAD_AMOUNT                           365
BAD_PRICE                             46
BAD_BCODE|BAD_PRICE|BAD_AMOUNT        16
BAD_BCODE|BAD_PRICE                    8
BAD_PRICE|BAD_AMOUNT                   3
Name: count, dtype: Int64


In [22]:
hq_sales_enriched = enrich_sales_with_last_purchase_cost(
    hq_sales_flagged,
    purchase_lines,
)

qc_unknown(hq_sales_enriched, "before refill")

hq_sales_enriched = refill_last_cost_from_icmas(
    data,
    hq_sales_enriched,
    last_cost_col="LAST_PURCHASE_COST",
)

qc_unknown(hq_sales_enriched, "after refill")

[before refill] UNKNOWN: 54,931 / 356,483 (15.41%)
[after refill] UNKNOWN: 54,931 / 356,483 (15.41%)


In [23]:
syp_sales_enriched = enrich_sales_with_last_purchase_cost(
    syp_sales_flagged,
    purchase_lines,
)

qc_unknown(syp_sales_enriched, "before refill")

syp_sales_enriched = refill_last_cost_from_icmas(
    data,
    syp_sales_enriched,
    last_cost_col="LAST_PURCHASE_COST",
)

qc_unknown(syp_sales_enriched, "after refill")

[before refill] UNKNOWN: 1,357 / 35,926 (3.78%)
[after refill] UNKNOWN: 1,357 / 35,926 (3.78%)


In [24]:
hq_sales_enriched["BRANCH"] = "HQ"
syp_sales_enriched["BRANCH"] = "SYP"

In [25]:
hq_sales_enriched["ID"] = (
    hq_sales_enriched["ID"].astype(str).str.strip()
    + "_"
    + hq_sales_enriched["BRANCH"].astype(str).str.strip().str.upper()
)

syp_sales_enriched["ID"] = (
    syp_sales_enriched["ID"].astype(str).str.strip()
    + "_"
    + syp_sales_enriched["BRANCH"].astype(str).str.strip().str.upper()
)

In [26]:
sales_all = pd.concat([hq_sales_enriched, syp_sales_enriched], ignore_index=True)

sales_all["BRANCH"] = sales_all["BRANCH"].astype("string")
sales_all["LAST_PURCHASE_COST"] = pd.to_numeric(sales_all["LAST_PURCHASE_COST"], errors="coerce")
sales_all["BILLDATE"] = pd.to_datetime(sales_all["BILLDATE"], errors="coerce")

sales_all["BRANCH_BILLNO"] = sales_all["BRANCH"] + "-" + sales_all["BILLNO"].astype(str)

In [27]:
sales_all["BILLTYPE_STD"] = (
    sales_all["BILLNO"]
    .astype("string")
    .str.upper()
    .str.replace(r"^3", "", regex=True)   # remove leading 3 if exists
    .str.extract(r"^(TFV|TAD|TAR|TR|TD|TF|CN|DN)", expand=False)
    .fillna("UNKNOWN")
)

In [28]:
KEEP_COLS = ['ID',
    'BILLDATE', 'BILLTYPE', 'JOURMODE',
    'BILLNO', 'BCODE', 'DETAIL',
    'STATUS', 'ISVAT',
    'QTY', 'UI', 'MTP', 'PRICE', 'XPRICE', 'DISCNT1', 'DISCNT2', 'DISCNT3',
    'DISCNT4', 'DED', 'VAT', 'AMOUNT', 'ACCTNO', 'PAID',
    'ACCT_NO', 'DONE', 'CANCELED',
    'PRICE_NUM', 'AMOUNT_NUM', 'IS_VALID', 'INVALID_REASON', 'ROW_ID',
    'LAST_PURCHASE_DATE', 'LAST_PURCHASE_COST', 'COST_STATUS',
    'BRANCH', 'BRANCH_BILLNO', 'BILLTYPE_STD'
]

# keep only columns that actually exist (prevents KeyError)
sales_all = sales_all[[c for c in KEEP_COLS if c in sales_all.columns]].copy()

In [29]:
dims = build_all_dims(sales_all)
{k: v.shape for k, v in dims.items()}

{'dim_date': (875, 8),
 'dim_product': (24544, 6),
 'dim_category': (36, 2),
 'dim_account': (1858, 4),
 'dim_branch': (2, 3),
 'dim_billtype': (9, 2)}

In [30]:
sales_all["ID"].is_unique

True

In [31]:
sales_all = filter_last_year_from_latest(sales_all, "BILLDATE")

In [32]:
import pandas as pd

# Make sure missing values are real nulls
sales_all = sales_all.where(pd.notna(sales_all), None)

# Convert datetime columns to strings (or dates)
date_cols = ["BILLDATE", "LAST_PURCHASE_DATE"]
for c in date_cols:
    if c in sales_all.columns:
        sales_all[c] = pd.to_datetime(sales_all[c], errors="coerce").dt.strftime("%Y-%m-%d")
        sales_all[c] = sales_all[c].where(sales_all[c].notna(), None)

In [33]:
# Fact sales lines
result = bulk_upsert_via_stage(
    conn,
    sales_all,
    target_table=f"fact_sales_lines",
    pk_cols=["ID"],
)

print(result)

{'rows_in_df': 229154, 'target_table': 'public.fact_sales_lines', 'stage_table': 'public.fact_sales_lines_stage'}


In [34]:
import pandas as pd
from sqlalchemy import text

def truncate_and_reload_dim(
    engine,
    table_name: str,
    df: pd.DataFrame,
    *,
    schema: str = "public",
    chunksize: int = 5000,
) -> int:
    """
    TRUNCATE target table then load df into it (full replace).

    Returns number of rows written.
    """
    if df is None:
        raise ValueError("df is None")
    if df.empty:
        raise ValueError("df is empty (refusing to TRUNCATE and load nothing)")

    full_name = f'{schema}."{table_name}"' if table_name.lower() != table_name else f"{schema}.{table_name}"

    # Use one transaction so you don't end up with empty table if load fails
    with engine.begin() as conn:
        conn.execute(text(f"TRUNCATE TABLE {full_name};"))

        # Load data
        df.to_sql(
            table_name,
            con=conn,              # important: use same transaction connection
            schema=schema,
            if_exists="append",
            index=False,
            method="multi",
            chunksize=chunksize,
        )

    return int(len(df))

In [35]:
import io
import pandas as pd

def truncate_and_copy_df(
    engine,
    table_name: str,
    df: pd.DataFrame,
    *,
    schema: str = "public",
    null_str: str = r"\N",     # Postgres default NULL marker for COPY
) -> int:
    """
    TRUNCATE table then COPY df into it using psycopg2 COPY for speed.
    Assumes df columns match the table columns by name.

    Returns number of rows loaded.
    """
    if df is None:
        raise ValueError("df is None")
    if df.empty:
        raise ValueError("df is empty (refusing to TRUNCATE and load nothing)")

    # --- IMPORTANT: make a safe copy & normalize missing values ---
    out = df.copy()

    # Convert pandas missing values to None so they become NULL in CSV
    out = out.where(pd.notna(out), None)

    # If you have datetime columns and your SQL column is DATE/TEXT, convert safely
    for col in out.columns:
        if pd.api.types.is_datetime64_any_dtype(out[col]):
            out[col] = out[col].dt.strftime("%Y-%m-%d")

    # Build CSV in memory
    buf = io.StringIO()
    out.to_csv(buf, index=False, header=True, na_rep=null_str)
    buf.seek(0)

    full_table = f'{schema}.{table_name}'
    cols_sql = ", ".join([f'"{c}"' for c in out.columns])  # quote columns safely

    copy_sql = f'''
        COPY {full_table} ({cols_sql})
        FROM STDIN WITH (FORMAT CSV, HEADER TRUE, DELIMITER ',', NULL '{null_str}')
    '''

    # Use a single DB transaction
    conn = engine.raw_connection()
    try:
        cur = conn.cursor()
        cur.execute(f"TRUNCATE TABLE {full_table};")
        cur.copy_expert(copy_sql, buf)
        conn.commit()
    except Exception:
        conn.rollback()
        raise
    finally:
        conn.close()

    return int(len(out))

In [36]:
truncate_and_copy_df(engine_supabase, "dim_date", dims["dim_date"])
truncate_and_copy_df(engine_supabase, "dim_account", dims["dim_account"])
truncate_and_copy_df(engine_supabase, "dim_product", dims["dim_product"])

24544

In [37]:
truncate_and_reload_dim(engine_supabase, "dim_branch", dims["dim_branch"], chunksize=100)
truncate_and_reload_dim(engine_supabase, "dim_billtype", dims["dim_billtype"], chunksize=100)
truncate_and_reload_dim(engine_supabase, "dim_category", dims["dim_category"], chunksize=500)

36

In [38]:
print("======== START ETL PARTS9 ========")

======== START ETL PARTS9 ========
